# CPSC 368 Data Cleaning Notebook (KNM Neighbours)

## Loading Data and Packages

In [1]:
import numpy as np
import pandas as pd

## KFF Data Cleaning

There are 3 KFF datasets: one for all adults aged 19-64, and two for males and females aged 19-64. The data is cleaned to only show the table itself, with other miscellaneous information removed.

In [2]:
KFF2019_adult_original = pd.read_csv('final_datasets_V1/KFF/kff_health_insurance_2019_adult.csv', skiprows = 2, nrows = 53).drop(columns='Footnotes', axis=1)
KFF2019_female_original = pd.read_csv('final_datasets_V1/KFF/kff_health_insurance_2019_female.csv', skiprows = 2, nrows = 53).drop(columns='Footnotes', axis=1)
KFF2019_male_original = pd.read_csv('final_datasets_V1/KFF/kff_health_insurance_2019_male.csv', skiprows = 2, nrows = 53).drop(columns='Footnotes', axis=1)

In [3]:
KFF2019_adult_original.to_csv("final_datasets_V1/cleaned/KFF2019_adult.csv", index=False)
KFF2019_female_original.to_csv("final_datasets_V1/cleaned/KFF2019_female.csv", index=False)
KFF2019_male_original.to_csv("final_datasets_V1/cleaned/KFF2019_male.csv", index=False)

In [4]:
for i in [KFF2019_adult_original, KFF2019_female_original, KFF2019_male_original]: 
    display(i.info())
    display(i.describe())
    display(i.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   53 non-null     object 
 1   Employer   53 non-null     float64
 2   Non-Group  53 non-null     float64
 3   Medicaid   53 non-null     float64
 4   Medicare   53 non-null     float64
 5   Military   53 non-null     float64
 6   Uninsured  53 non-null     float64
 7   Total      53 non-null     float64
dtypes: float64(7), object(1)
memory usage: 3.4+ KB


None

,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
count,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.0
mean,0.615604,0.076075,0.149623,0.021660,0.018094,0.118849,1.0
std,0.066573,0.018413,0.063945,0.007643,0.008998,0.043453,0.0
min,0.305000,0.036000,0.061000,0.009000,0.005000,0.044000,1.0
25%,0.584000,0.068000,0.101000,0.016000,0.011000,0.087000,1.0
50%,0.618000,0.073000,0.141000,0.020000,0.018000,0.114000,1.0
75%,0.658000,0.081000,0.182000,0.025000,0.022000,0.149000,1.0
max,0.704000,0.131000,0.429000,0.042000,0.047000,0.245000,1.0


Location     0
Employer     0
Non-Group    0
Medicaid     0
Medicare     0
Military     0
Uninsured    0
Total        0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   53 non-null     object 
 1   Employer   53 non-null     float64
 2   Non-Group  53 non-null     float64
 3   Medicaid   53 non-null     float64
 4   Medicare   53 non-null     float64
 5   Military   52 non-null     float64
 6   Uninsured  53 non-null     float64
 7   Total      53 non-null     float64
dtypes: float64(7), object(1)
memory usage: 3.4+ KB


...

Location     0
Employer     0
Non-Group    0
Medicaid     0
Medicare     0
Military     1
Uninsured    0
Total        0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   53 non-null     object 
 1   Employer   53 non-null     float64
 2   Non-Group  53 non-null     float64
 3   Medicaid   53 non-null     float64
 4   Medicare   53 non-null     float64
 5   Military   53 non-null     float64
 6   Uninsured  53 non-null     float64
 7   Total      53 non-null     float64
dtypes: float64(7), object(1)
memory usage: 3.4+ KB


None

,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
count,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.0
mean,0.620698,0.074415,0.128623,0.022094,0.019377,0.134623,1.0
std,0.067915,0.018067,0.060703,0.008383,0.007860,0.045389,0.0
min,0.304000,0.037000,0.051000,0.010000,0.006000,0.056000,1.0
25%,0.586000,0.065000,0.083000,0.017000,0.013000,0.101000,1.0
50%,0.629000,0.071000,0.124000,0.021000,0.020000,0.126000,1.0
75%,0.667000,0.078000,0.159000,0.026000,0.025000,0.167000,1.0
max,0.717000,0.123000,0.404000,0.047000,0.046000,0.259000,1.0


Location     0
Employer     0
Non-Group    0
Medicaid     0
Medicare     0
Military     0
Uninsured    0
Total        0
dtype: int64

## U.S. Chronic Disease Indicators

The U.S. Chronic Disease Indicators dataset contains many types of data for a variety of topics, and given our topic questions, we will need to filter the data to focus on the topics of 'Cardiovascular Disease' and 'Cancer'. Any NA values are filled with 0 under the assumption that said data could not be collected at all.

In [5]:
USCDI_original = pd.read_csv('final_datasets_V1/CDC/U.S._Chronic_Disease_Indicators.csv')

In [6]:
condition_0_topic = USCDI_original["Topic"].isin(['Cardiovascular Disease', 'Cancer'])
condition_0_dvu = USCDI_original["DataValueUnit"].isin(['cases per 100,000', 'per 100,000'])
condition_0_sc1 = USCDI_original["StratificationCategory1"].isin(["Sex", "Age", "Overall"])
USCDI_filter = USCDI_original[condition_0_topic & condition_0_dvu & condition_0_sc1][["YearStart","YearEnd","LocationDesc",
                                                                                      "Topic","Question","DataValueUnit", 
                                                                                      "DataValueType", "DataValue",
                                                                                      "StratificationCategory1", "Stratification1"]].fillna(0)
USCDI_filter.to_csv('final_datasets_V1/cleaned/USCDI_filter.csv', index=False)

In [7]:
display(USCDI_filter.info())
display(USCDI_filter.describe())
display(USCDI_filter.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 8592 entries, 115 to 274446
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   YearStart                8592 non-null   int64  
 1   YearEnd                  8592 non-null   int64  
 2   LocationDesc             8592 non-null   object 
 3   Topic                    8592 non-null   object 
 4   Question                 8592 non-null   object 
 5   DataValueUnit            8592 non-null   object 
 6   DataValueType            8592 non-null   object 
 7   DataValue                8592 non-null   float64
 8   StratificationCategory1  8592 non-null   object 
 9   Stratification1          8592 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 738.4+ KB


None

,YearStart,YearEnd,DataValue
count,8592.000000,8592.000000,8592.000000
mean,2017.706006,2019.745112,128.980575
std,2.348503,0.718885,188.041781
min,2015.000000,2019.000000,0.000000
25%,2015.000000,2019.000000,17.700000
50%,2016.000000,2020.000000,47.700000
75%,2020.000000,2020.000000,167.625000
max,2021.000000,2021.000000,1456.500000


YearStart                  0
YearEnd                    0
LocationDesc               0
Topic                      0
Question                   0
DataValueUnit              0
DataValueType              0
DataValue                  0
StratificationCategory1    0
Stratification1            0
dtype: int64